## Lagragien for Static Balance

*Source : Dynamics and Control of np.single-Line Kites - Gonzalo Sanchez-Arrlaga*

*Goal : Find the Angle of Attack ($\alpha$) and the Elevation($\beta$) in Regard of Wind Speed*

## Import

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Data

In [31]:
# # %% Xfoil Polar 
# polar_data = pd.read_csv(os.path.join(os.getcwd(),"polar_SK50VD.csv")) # Same airfoil as 10VB
polar_data = pd.read_csv(filepath_or_buffer='polar_SK50VD.csv') # Same airfoil as 10VB
polar = {
    "alpha" : polar_data.alpha,
    "CL" : polar_data.CL,
    "CD" : polar_data.CD, 
    "CDp" : polar_data.CDp,
    "CM" : polar_data.CM,
    "Top_Xtr" : polar_data.Top_Xtr,
    "Bot_Xtr" : polar_data.Bot_Xtr,
}

# # %% Air
g = 9.8 
rho = 1.225

# # %% Kite
Surface = 10 # (m2)
mass = 5.5 # (kg)
height = 2.2 # distance from towpoint to profile (m)
corde = 1.99 # (m)
Xt = 0.3* corde # towpoint (m)
Xg = 0.5 * corde  # mass center (m)
delta_deg = 83 # Bridle Angle (°)
r = 2.26 # Bridle Length |QG| (m)

# %% Calculs 
polar["Xcop"] = (0.25 - polar["CM"]/polar["CL"])* corde
polar["Xcp"] = Xg-polar["Xcop"]
polar["CN"] = (polar["CL"]*polar["CL"]+ polar["CD"]*polar["CD"])**2

delta = delta_deg*np.pi/180 #atan(height / (Xg-Xt))
polar["sigma"] = polar["Xcp"] / r 

## Main

In [33]:
# %% Input
W0 =5 # Wind speed (m/s)

# %% Equation 1 
beta = rho * Surface * W0**2 / (2 * mass * g)

polar["cos1"] = np.cos(delta - polar["alpha"]*np.pi/180)
polar["cos2"] = - beta * (polar["sigma"] - np.cos(delta))* polar["CN"]

polar["sum"] = polar["np.cos1"] + polar["np.cos2"]

# %% Alpha
polar_sum = sns.lineplot(data=polar, x="alpha", y="sum")
polar_sum.set_title("equation1")
polar_sum.set_ylabel('sum')
polar_sum.set_xlabel('alpha')
plt.show()

# %% Polar theta
polar["cos"] = np.cos(- beta * (polar["sigma"] - np.cos(delta))*polar["CN"])*180/np.pi
polar["theta1"] = delta_deg - polar["cos"]
polar["theta2"] = -(delta_deg + polar["cos"] - 180)
polar["diff1"] = polar["theta1"] - polar["alpha"]
polar["diff2 "]= polar["theta2"] - polar["alpha"]

# %% Equation 2 
polar["gamma"] = np.atan((beta * polar["CN"] *np.cos(polar["alpha"]*np.pi/180)-1) / (beta * polar["CN"] * np.sin(polar["alpha"]*np.pi/180))  ) *180/np.pi

# %% regression de l'élévation en fonction de l'incidence : 
P = np.polyfit(np.array(polar["alpha"]), polar["gamma"], 1)
polar["mdl"] = P[0]*np.array(polar["alpha"]) + P[1]
R = np.corrcoef(np.array(polar["alpha"]), polar["gamma"])

polar_gamma = sns.lineplot(data=polar, x="alpha", y="gamma", style='*', legend="full")
polar_mdl = sns.lineplot(data=polar, x="alpha", y="mdl", style='r')
# polar_gamma.set_title('regression linéaire')
# polar_gamma.set_ylabel('gamma')
# polar_gamma.set_xlabel('alpha')
# polar_gamma.legend('données', 'ligne de regression')
plt.show()

# Coefficients du modèle
intercept = P[1]
slope = P[0]

# R²
r_squared = R**2

# Afficher les résultats
print('Equation: y = #.2f + #.2f*x\n', intercept, slope)
print('R²: #.2f\n', r_squared)

AttributeError: module 'numpy' has no attribute 'np'